#  Generating GRCh37 Medical Genes Benchmark
This notebook details the steps to generate the challenging medically relevant genes benchmark. All paths are from the top level directory of the repository and large dependency files that can not be stored in a git repo are indicated with URL in comments.

## Steps  

1. Look up coordinates for gene symbols in ENSEMBLE GRCh37 Human Genes v101 of the union of Mandelker et al Supplementary Table 13, COSMIC Cancer Gene Census, Steve Lincoln Medical Gene Lists -- `scripts\GRCh37_lookup_MRG_symbol_coordinates_ENSEMBL.R`

2. Find overlap of genes with HG002 v4.2.1, then add slop and find overlap with HG002 trio-hifiasm (v0.11) diploid assembly confident regions `HG002v11-align2-GRCh37.dip.bed`

3. Find genes with < 90% included bases in v4.2.1 HG002 small variant benchmark and fully covered with overlapping segdups and flanking sequence in HG002 hifiasm v0.11 GRCh37 dip.bed, find union of GRCh37 and GRCh38 MRG lists, then add genes that are unique to GRCh37 but still fully covered with overlapping segdups and flanking sequence in HG002 hifiasm v0.11 GRCh37 dip.bed -- `scripts/find_coordinates_of_MRG_GRCh37_GRCh38_union.R`

4. Use coordinates for benchmark then remove
    - homopolymers and imperfect homopolymers > 20
    - SVs with 50bp flanking and overlapping tandem repeats
    - hifiasm error
    - GRCh37 GAPs
    - Remove partially covered tandem repeats
    - Remove MHC region
    
5. Generate stratification files for Complex Variants in Tandem Repeats
    - GRCh37_MRG_stratification_ComplexVar_in_TR.bed

## Find overlap with v4.2.1 in ENSEMBL gene annotations

`HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed` is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/AshkenazimTrio/HG002_NA24385_son/NISTv4.2.1/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed

`GRCh37_ENSEMBL_MRG_coordinates.bed` is the output of `scripts/GRCh37_lookup_MRG_symbol_coordinates_ENSEMBL.R`

Need to then apply https://gitlab.nist.gov/gitlab/nolson/mrg-bench-manuscript/-/blob/master/data/gene_coords/ensembl_coords/selected_coordinates_for_duplicated_gene_symbol_entries.tsv which yields https://gitlab.nist.gov/gitlab/nolson/mrg-bench-manuscript/-/blob/master/data/gene_coords/unsorted/GRCh37_mrg_bench_gene.bed

In [ ]:

bedtools coverage \
    -a data/manually_created_files/GRCh37_mrg_full_gene.bed \
    -b data/v4.2.1_benchmark_regions/HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed \
    | cut -f1,2,3,4,7,8  \
    > workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_overlap_with_v4.2.1_benchmark.bed

## Create flanking sequence bed for MRG candidates

In [ ]:
bedtools slop \
    -i data/manually_created_files/GRCh37_mrg_full_gene.bed \
    -g resources/human.b37.genome \
    -b 20000 \
    > workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_slop20000bp.bed


## Find breaks in dip.bed in MRG candidates

`HG002v11-align2-GRCh37.dip.bed` is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_benchmark_v0.02/GRCh37/hifiasm_v0.11/HG002v11-align2-GRCh37.dip.bed

`HG002v11-align2-GRCh37.dip_check_for_breaks.bed` is generated as HG002v11-align2-GRCh37.dip.bed chrom, start, start+1

In [ ]:
python scripts/find_overlap_per_gene.py \
    --input_benchmark data/manually_created_files/HG002v11-align2-GRCh37.dip_check_for_breaks.bed \
    --input_genes workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_slop20000bp.bed \
    --output  workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_slop20000bp_check_for_breaks_in_dip.bed

## Find coordinates of ENSEMBL gene annotations with flanking sequence and overlapping segdups

In [ ]:
python scripts/expand_gene_coordinates_with_flank_and_overlapping_segdups_GRCh37.py \
    --input_genes workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_slop20000bp.bed \
    --output workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_slop20000bp_flanking_and_segdups_coordinates.bed

## Append gene names 
Adding genes names column from `GRCh37_mrg_full_gene_coordinates_slop20000bp.bed` to `GRCh37_mrg_full_gene_coordinates_slop20000bp_flanking_and_segdups_coordinates.bed`.


In [ ]:
cat workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_slop20000bp.bed \
    | cut -f4 \
    | paste workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_slop20000bp_flanking_and_segdups_coordinates.bed - \
    > workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_slop20000bp_flanking_and_segdups_coordinates_w_gene_names.bed 

## Sort HG002v11-align2-GRCh37.dip.bed

In [ ]:
cat data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.bed \
    | sort -k1,1 -k2,2n \
    > workflow/smallvar_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_sorted.bed

## Find overlap of HG002 GRCh37 hifiasm v0.11 of ENSEMBL gene annotations with flanking sequence and overlapping segdups

In [ ]:
bedtools coverage \
    -a workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_slop20000bp_flanking_and_segdups_coordinates_w_gene_names.bed \
    -b workflow/smallvar_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_sorted.bed \
    | cut -f1,2,3,4,7,8 \
    > workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_slop20000bp_flanking_and_segdups_coordinates_w_gene_names_overlap_with_HG002v11-align2-GRCh37.dip_sorted.bed


## Create HG002_GRCh37_overlap_v4.2.1_and_hifiasm.tsv 
Combine chrom, start, end, gene name, bp_covered, frac_covered columns of `GRCh37_mrg_full_gene_coordinates_overlap_with_v4.2.1_benchmark.bed`, appending columns 5 and 6 of `GRCh37_mrg_full_gene_coordinates_slop20000bp_flanking_and_segdups_coordinates_w_gene_names_overlap_with_HG002v11-align2-GRCh37.dip_sorted.bed`, then append column 5 of `GRCh37_mrg_full_gene_coordinates_slop20000bp_check_for_breaks_in_dip.bed` 

`GRCh37_overlap_v4.2.1_and_hifiasm.tsv` column names are chrom, start, end, gene, bp_overlap_v4.2.1, percent_overlap_v4.2.1, bp_flanking_plus_segdups_overlap_hifiasm, percent_flanking_plus_segdups_overlap_hifiasm, flanking_breaks_in_dip_bed


In [ ]:
cat workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_slop20000bp_flanking_and_segdups_coordinates_w_gene_names_overlap_with_HG002v11-align2-GRCh37.dip_sorted.bed \
    | cut -f5,6 \
    | paste workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_overlap_with_v4.2.1_benchmark.bed - \
    > workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_overlap_with_v4.2.1_benchmark_overlap_hifiasm.bed


cat workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_slop20000bp_check_for_breaks_in_dip.bed \
    | cut -f5 \
    | paste workflow/smallvar_benchmark/GRCh37/GRCh37_mrg_full_gene_coordinates_overlap_with_v4.2.1_benchmark_overlap_hifiasm.bed - \
    > workflow/smallvar_benchmark/GRCh37/GRCh37_overlap_v4.2.1_and_hifiasm_temp.bed

cat data/manually_created_files/GRCh37_overlap_v4.2.1_and_hifiasm_header.tsv \
    > workflow/smallvar_benchmark/GRCh37/GRCh37_overlap_v4.2.1_and_hifiasm.tsv

cat workflow/smallvar_benchmark/GRCh37/GRCh37_overlap_v4.2.1_and_hifiasm_temp.bed \
    >> workflow/smallvar_benchmark/GRCh37/GRCh37_overlap_v4.2.1_and_hifiasm.tsv


##  Use find_coordinates_of_MRG_GRCh37_GRCh38_union.R to generate HG002_GRCh37_CMRG_coordinates.bed

NOTE: find_coordinates_of_MRG_GRCh37_GRCh38_union.R depends on the creation of workflow/smallvar_benchmark/GRCh37/GRCh38_overlap_v4.2.1_and_hifiasm.tsv as detailed in analysis/GRCh38_HG002_medical_genes_benchmark_generation.ipynb

## Run bedtools merge

In [ ]:
bedtools merge \
    -i workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates.bed \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_bedtools_merge.bed

# From these genes to be benchmarked remove the following regions that we exclude from the diploid assembly based variant calls:

    - homopolymers and imperfect homopolymers > 20
    - SVs with 50bp flanking and overlapping tandem repeats
    - hifiasm error
    - GRCh37 GAPs
    - Remove partially covered tandem repeats
    
    

## Generate GRCh37_SimpleRepeat_homopolymer_gt20.bed and GRCh37_SimpleRepeat_imperfecthomopolgt20_slop5.bed

NOTE: This is included for documentation of workflow purposes and is not recommended to be run. The output files are available in workflow/cmrg_stratifications/GRCh37/ and used in the rest of this notebook from that location

`findSimpleRegions_quad.py` is from: https://opendata.nist.gov/pdrsrv/mds2-2190/GRCh38/LowComplexity/findSimpleRegions_quad.py

`hs37d5.fa` can be retrieved from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/references/GRCh37/hs37d5.fa.gz

In [ ]:
python scripts/findSimpleRegions_quad.py \
	-p 20 -d 100000 -t 100000 -q 100000 \
	resources/hs37d5.fa workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_p20.bed

cat workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_p20.bed \
	| grep "^[0-9XY]" \
	| grep -v '_' \
	| sed 's/^X/23/;s/^Y/24/' \
	| sort -k1,1n -k2,2n -k3,3n \
	| sed 's/^23/X/;s/^24/Y/' \
	| slopBed -i stdin -b 5 -g resources/human.b37.genome \
	| bgzip -c \
	> workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_homopolymer_gt20_slop5.bed.gz

python scripts/findSimpleRegions_quad.py \
	-p 3 -d 100000 -t 100000 -q 100000 \
	resources/hs37d5.fa workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_p3.bed


grep 'unit=C' workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_p3.bed \
	| mergeBed -i stdin -d 1 \
	| awk '$3-$2>20' \
	> workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_imperfecthomopolgt20_C.bed

grep 'unit=G' workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_p3.bed \
	| mergeBed -i stdin -d 1 \
	| awk '$3-$2>20' \
	> workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_imperfecthomopolgt20_G.bed

grep 'unit=A' workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_p3.bed \
	| mergeBed -i stdin -d 1 \
	| awk '$3-$2>20' \
	> workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_imperfecthomopolgt20_A.bed

grep 'unit=T' workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_p3.bed \
	| mergeBed -i stdin -d 1 \
	| awk '$3-$2>20' \
	> workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_imperfecthomopolgt20_T.bed

multiIntersectBed -i workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_imperfecthomopolgt20_C.bed \
	workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_imperfecthomopolgt20_G.bed \
	workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_imperfecthomopolgt20_A.bed \
	workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_imperfecthomopolgt20_T.bed | 
    cut -f1-3 | grep "^[0-9XY]" | grep -v '_' |
	slopBed -i stdin -b 5 -g resources/human.b37.genome |
	sed 's/^X/23/;s/^Y/24/' | 
	sort -k1,1n -k2,2n -k3,3n | 
	sed 's/^23/X/;s/^24/Y/' | 
	mergeBed -i stdin | 
	bgzip -c \
	> workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_imperfecthomopolgt20_slop5.bed.gz

## Remove homopolymers > 20bp

In [ ]:
bedtools subtract \
    -a workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_bedtools_merge.bed \
    -b workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_homopolymer_gt20_slop5.bed.gz \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_SimpleRepeat_homopolymer_gt20_slop5.bed

## Remove imperfect homopolymers > 20bp

In [ ]:
bedtools subtract \
    -a workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_SimpleRepeat_homopolymer_gt20_slop5.bed \
    -b workflow/cmrg_stratifications/GRCh37/GRCh37_SimpleRepeat_imperfecthomopolgt20_slop5.bed.gz \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_SimpleRepeat_imperfecthomopolgt20_slop5.bed

## SVs with 50bp flanking and overlapping tandem repeats

`HG002v11-align2-GRCh37.dip.vcf.gz` is retrieved from ftp://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_benchmark_v0.01.00/GRCh37/hifiasm_v0.11/HG002v11-align2-GRCh37.dip.vcf.gz

`GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz` is from ftp://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/genome-stratifications/v2.0/GRCh38/LowComplexity/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz

In [ ]:
gunzip \
    -c data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    | awk 'length($4)>49 || length($5)>49' \
    | awk '{FS=OFS="\t"} {print $1,$2-1,$2+length($4)}' \
    > workflow/smallvar_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp.bed

intersectBed \
    -wa -a resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
    -b workflow/smallvar_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp.bed \
    | multiIntersectBed -i stdin workflow/smallvar_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp.bed \
    |  awk '{FS=OFS="\t"} {print $1,$2-50,$3+50}' \
    | mergeBed -i stdin -d 1000 
    > workflow/smallvar_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000.bed

bedtools subtract \
    -a workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_SimpleRepeat_imperfecthomopolgt20_slop5.bed \
    -b workflow/smallvar_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000.bed \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_SVsgt49bp_repeatexpanded_slop50_merge1000.bed

## Remove hifiasm error on chr21
`GRCh37_hifiasm_error.bed`
 was created through manual curation of clusters of errors identified during evaluation steps of benchmark development

In [ ]:
bedtools subtract \
    -a workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_SVsgt49bp_repeatexpanded_slop50_merge1000.bed \
    -b data/manually_created_files/GRCh37_hifiasm_error.bed \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_hifiasm_error.bed

## Remove GRCh37 GAPs
`GRCh37_MRG_GAPs.bed` was created through manual curation of clusters of errors identified during evaluation steps of benchmark development


In [ ]:
bedtools subtract \
    -a workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_hifiasm_error.bed \
    -b data/manually_created_files/GRCh37_MRG_GAPs.bed \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_MRG_GAPs.bed

## Sort

In [ ]:
cat workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_MRG_GAPs.bed \
    | sort -k1,1n -k2,2n \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_MRG_GAPs_sorted.bed

## Remove partially covered tandem repeats

In [ ]:
complementBed \
    -i workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_MRG_GAPs_sorted.bed \
    -g resources/human.b37.genome \
    | intersectBed \
        -wa -a resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
        -b stdin \
    | subtractBed \
        -a workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_MRG_GAPs_sorted.bed \
        -b stdin \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_partial_tandem_repeats.bed

## Prepare benchmark VCF

__Notes__
1. `workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark.vcf.gz` is  https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_benchmark_v0.02/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz after updates to the headers

2. `workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark.bed` is https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_benchmark_v0.02/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.bed

In [ ]:
cp workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_partial_tandem_repeats.bed \
    workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark.bed

cat workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark.bed \
    | awk '{sum+=$3-$2} END {print sum}'

bedtools intersect \
    -a data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    -b workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_bedtools_merge.bed \
    -header \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark.vcf

bgzip workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark.vcf

tabix workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark.vcf.gz

## SV Benchmark

In [ ]:
# Find SVs MRG benchmark gene coordinates
bedtools intersect \
    -a workflow/smallvar_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000.bed \
    -b workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_bedtools_merge.bed \
    > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates.bed

# Subset to SVs only gt49bp 
gunzip \
    -c data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    | awk '{FS="\t|,"} {if($1 ~ /^#/ || length($4)-length($5)>49 || length($5)-length($4)>49 || length($6)-length($4)>49) print}' \
    | intersectBed \
        -a workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates.bed \
        -b stdin -c \
    | awk '$4>0' \
    | cut -f1-3 \
    > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp.bed 

# Find isolated SVs  

gunzip \
    -c data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    | awk '{FS="\t|,"} {if($1 ~ /^#/ || length($4)-length($5)>9 || length($5)-length($4)>9 || length($6)-length($4)>9) print}' \
    | intersectBed -a workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp.bed \
    -b stdin -c \
    | awk '$4==1' \
    | cut -f1-3 \
    > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp_isolated.bed 

# Find complex SVs  

gunzip \
    -c data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    | awk '{FS="\t|,"} {if($1 ~ /^#/ || length($4)-length($5)>9 || length($5)-length($4)>9 || length($6)-length($4)>9) print}' \
    | intersectBed \
        -a workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp.bed \
        -b stdin -c \
        | awk '$4>1' \
        | cut -f1-3 \
        > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp_complexSVs.bed 

# Remove complex SVs from MRG gene candidate coordinates and remove GAPs 
bedtools subtract \
    -a workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_bedtools_merge.bed \
    -b workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp_complexSVs.bed \
    | bedtools subtract \
        -a stdin 
        -b  data/manually_created_files/GRCh37_MRG_GAPs.bed \
    > workflow/SV_benchmark/GRCh37/HG002_GRCh37_MRG_draft_SV_benchmark_temp.bed

#HG002v11-align2-GRCh37.dip_complexindelsgt9bpinRepeats.bed  from the SV benchmark bed:
# Find tandem repeats and homopolymers that have multiple indels >9bp, since these can add up to >49bp and should be subtracted from the benchmark SV bed
gunzip \
    -c data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    | awk '{FS="\t|,"} {if($1 ~ /^#/ || length($4)-length($5)>9 || length($5)-length($4)>9 || length($6)-length($4)>9) print}' \
    | intersectBed \
        -a resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
        -b stdin -c \
    | awk '$4>1' \
    | cut -f1-3 \
    | intersectBed \
        -v -a stdin \
        -b workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp.bed \
        > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_complexindelsgt9bpinRepeats.bed

bedtools subtract \
    -a workflow/SV_benchmark/GRCh37/HG002_GRCh37_MRG_draft_SV_benchmark_temp.bed \
    -b workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_complexindelsgt9bpinRepeats.bed \
    > workflow/SV_benchmark/GRCh37/HG002_GRCh37_MRG_draft_SV_benchmark.bed

cat workflow/SV_benchmark/GRCh37/HG002_GRCh37_MRG_draft_SV_benchmark.bed \
    | awk '{sum+=$3-$2} END {print sum}'

# Decompose for truvari comparison
#vt decompose -s HG002v11-align2-GRCh37.dip.vcf -o HG002v11-align2-GRCh37.dip_decomposed.vcf
#python script to remove ambiguous (non-ACTGN) REF

gunzip \
    -c data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    > data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf


## remove ambiguous (non-ACGTN) in REF field. Adjust path to where you keep this file
python scripts/fix_reference_allele.py \
    --input_vcf_file data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf \
    --output_file workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig.dip.vcf

rm data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf

## zip for bcftools
bgzip \
    -c workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig.dip.vcf \
    > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig.dip.vcf.gz

tabix workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig.dip.vcf.gz

## split multiallelic to biallelic
bcftools norm -m- \
    -Oz workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig.dip.vcf.gz \
    -o workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig_norm_m.vcf.gz

## left align and normalize indels. 
bcftools norm \
    -f resources/hs37d5.fa \
    -Oz -o workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig_norm_mf.vcf.gz \
    workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig_norm_m.vcf.gz

## remove duplicate records
bcftools norm -d none \
    -Oz workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig_norm_mf.vcf.gz \
    -o workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig_norm_mfd.vcf.gz

## remove MHC region
bedtools subtract \
    -header -a workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig_norm_mfd.vcf.gz \
    -b resources//giab_stratifications/GRCh37_MHC.bed.gz \
    | bgzip -c \
    > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig_norm_mfd_noMHC.vcf.gz

## intersect w/ benchmark bed and subset to >39bp in REF or ALT fields. 
#intersect w/ MRG target regions and subset >39 bp

bedtools intersect -header \
    -a workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig_norm_mfd_noMHC.vcf.gz \
    -b workflow/SV_benchmark/GRCh37/HG002_GRCh37_MRG_draft_SV_benchmark.bed \
    | awk '$1 ~ /^#/ || length($4)>39 || length($5)>39' \
    | bgzip -c \
    > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig_norm_mfd_noMHC_intersectBenchBED_gt39bp.vcf.gz

## index vcf, required by truvari
tabix -p vcf workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig_norm_mfd_noMHC_intersectBenchBED_gt39bp.vcf.gz


## Find benchmark variants between 35 and 49 base pairs in size and exclude overlapping tandem repeats plus slop 50bp on either side. Remove these from the benchmark regions bed so that it includes only SVs that are greater than 49 base pairs

python scripts/SVs_between_35_50bp.py \
    --input workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37._noambig_norm_mfd_noMHC_intersectBenchBED_gt39bp.vcf \
    --output workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.02_SVs_gt34_and_lt_50bp.vcf

# needs to be run in bedops_env
vcf2bed < workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.02_SVs_gt34_and_lt_50bp.vcf \
    > workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.02_SVs_gt34_and_lt_50bp.bed

intersectBed -wa \
    -a resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
    -b workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.02_SVs_gt34_and_lt_50bp.bed \
    | multiIntersectBed \
        -i stdin workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.02_SVs_gt34_and_lt_50bp.bed \
    |  awk '{FS=OFS="\t"} {print $1,$2-50,$3+50}' \
    > workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.02_gt34_and_lt_50bp_repeatexpanded_slop50.bed



bedtools subtract \
    -a workflow/SV_benchmark/GRCh37/HG002_GRCh37_MRG_draft_SV_benchmark.bed \
    -b workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.02_gt34_and_lt_50bp_repeatexpanded_slop50.bed \
    > workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.bed


## Prepare release benchmark files

Steps for creating excluding errors found in curation of the MRG benchmark, in order to create v1.0 bed files for small variants and SVs  

Updated 4/27/21 to exclude 50bp on either side of errors and unsure variants found in curation, because sometimes a deletion and overlapping SNVs outside a repeat weren't excluded from benchmark stats. Also, remove ".;" at beginning of INFO field introduced by svanalyzer svwiden. Produces v1.00.01 small variant and structural variant MRG benchmarks


In [ ]:
##errors to exclude found in curation of SVs
#only found 2 errors so manually created bed GRCh37_curation_medicalgene_SV_errorsorunsure.bed
#expand bed coordinates to completely cover any overlapping homopolymers and tandem repeats
intersectBed -wa \
    -a resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
    -b data/manually_created_files/GRCh37_curation_medicalgene_SV_errorsorunsure.bed \
    | multiIntersectBed -i stdin data/manually_created_files/GRCh37_curation_medicalgene_SV_errorsorunsure.bed \
    | mergeBed -i stdin \
    > workflow/release_benchmark_generation/GRCh37_curation_medicalgene_SV_errorsorunsure_repeatexpanded.bed

#used NCBI remap to generate GRCh38_curation_medicalgene_SV_errorsorunsure_repeatexpanded.bed


#create bed with sites curated as unsure or incorrect in the benchmark in GRCh37 coordinates
cut -f3,6,8,12,20 \
    data/manually_created_files/combined\ curation\ responses\ from\ benchmarking\ with\ sm\ variant\ v0.02.03\ -\ GRCh37andGRCh38.tsv \
    | grep 'sure\|o' \
    | grep -v ^ref \
    | awk '{FS=OFS="\t"} {print $2, $3-50, $3+length($4)+50}' \
    | sort -k1,1n -k2,2n -k3,3n \
    | mergeBed -i stdin -d 100 \
    > workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure.bed

intersectBed -wa \
    -a resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
    -b workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure.bed \
    | multiIntersectBed \
        -i stdin workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure.bed \
    | mergeBed -i stdin \
    > workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_repeatexpanded.bed

cd data/manually_created_files/
##for manuscript, identify number of errors/unsure identified in curation and the number of these that would have been excluded by curating common FPs/FNs first
#create bed with sites curated as unsure or incorrect in the benchmark in GRCh37 coordinates
grep ^Common combined\ curation\ responses\ from\ benchmarking\ with\ sm\ variant\ v0.02.03\ -\ GRCh37andGRCh38.tsv \
    | cut -f3,6,8,12,20 \
    | grep 'sure\|o' \
    | grep -v ^ref \
    | awk '{FS=OFS="\t"} {print $2, $3-50, $3+length($4)+50}' \
    | sort -k1,1n -k2,2n -k3,3n \
    | mergeBed -i stdin -d 100 \
    > ../../workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_Commononly.bed

grep -v ^Common combined\ curation\ responses\ from\ benchmarking\ with\ sm\ variant\ v0.02.03\ -\ GRCh37andGRCh38.tsv \
    | cut -f3,6,8,12,20 \
    | grep 'sure\|o' \
    | grep -v ^ref \
    | grep ^GRCh37 \
    | awk '{FS=OFS="\t"} {print $2, $3-50, $3+length($4)+50}' \
    | sort -k1,1n -k2,2n -k3,3n  \
    > ../../workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_evaluationonly.bed

cd ../..

wc -l workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_evaluationonly.bed
#      50


#expand bed coordinates to completely cover any overlapping homopolymers and tandem repeats
intersectBed -wa \
    -a resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
    -b workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_Commononly.bed \
    | multiIntersectBed \
        -i stdin workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_Commononly.bed \
    | mergeBed -i stdin \
    | intersectBed -v \
        -a workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_evaluationonly.bed \
        -b stdin \
    | wc -l
# 6

#download GRCh37 v0.02.03 from https://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_benchmark_v0.02/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.bed
#exclude errors/unsure sites found in SV curation, small variant curation, small variant complex TR curation, and FPs from complex TR comparison
subtractBed \
        -a data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.bed \
        -b workflow/release_benchmark_generation/GRCh37_curation_medicalgene_SV_errorsorunsure_repeatexpanded.bed \
    | subtractBed \
        -a stdin \
        -b workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_repeatexpanded.bed \
    | subtractBed \
        -a stdin \
        -b data/manually_created_files/GRCh37_curation_medicalgene_smallvar_complexrepeat_errorsorunsure_repeatexpanded.bed \
    | subtractBed \
        -a stdin \
        -b data/manually_created_files/HiCanu_2.1_HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03_intersected_FPs_repeatexpanded_slop50.bed \
        > workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v1.00.01.bed

#Make sure the bed size doesn't decrease more than expected
awk '{sum+=$3-$2} END {print sum}' \
    data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.bed 
#11712171
awk '{sum+=$3-$2} END {print sum}' \
    workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v1.00.01.bed 
#11679803

#Make sure the number of variants in the bed don't decrease more than expected
intersectBed -a data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz \
    -b data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.bed | wc -l
#21907
intersectBed -a data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz \
    -b workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.03.00.bed | wc -l
#21664
intersectBed -a data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz \
    -b workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v1.00.01.bed | wc -l
#21591

cp data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz \
    workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v1.00.01.vcf.gz 
cp data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz.tbi \
    workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v1.00.01.vcf.gz.tbi

#download GRCh37 v0.01 bed from https://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_SV_benchmark_v0.01/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.bed
#exclude errors/unsure sites found in SV curation
subtractBed \
    -a data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.bed \
    -b workflow/release_benchmark_generation/GRCh37_curation_medicalgene_SV_errorsorunsure_repeatexpanded.bed \
    > workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.02.00.bed

#Make sure the bed size doesn't decrease more than expected
awk '{sum+=$3-$2} END {print sum}' \
    data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.bed 
#11966919
awk '{sum+=$3-$2} END {print sum}' \
    workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.02.00.bed 
#11966249

#Make sure the number of variants in the bed don't decrease more than expected
intersectBed \
    -a data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.vcf.gz \
    -b data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.bed \
    | wc -l
#205

intersectBed 
    -a data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.vcf.gz \
    -b workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.02.00.bed \
    | wc -l
#204

cp data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.vcf.gz \
    workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.02.00.vcf.gz 
cp data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.vcf.gz.tbi \
    workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.02.00.vcf.gz.tbi 

#remove ".;" at beginning of INFO field introduced by svanalyzer svwiden
gunzip -c workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.02.00.vcf.gz \
    | sed 's/\.;REPTYPE/REPTYPE/' \
    | bgzip -c 
    > workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v1.00.01.vcf.gz

tabix workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v1.00.01.vcf.gz

cp workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.02.00.bed \
    workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v1.00.01.bed 


## Generate stratification files for Complex Variants in Tandem Repeats

`HG002v11-align2-GRCh37.dip.vcf` is from https://drive.google.com/drive/folders/1gUCmveutqniwPN8aZW-gzumrWsMHQdnb
`GRCh37_AllTandemRepeats_gt100bp_slop5.bed` is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/genome-stratifications/v2.0/GRCh37/LowComplexity/GRCh37_AllTandemRepeats_gt100bp_slop5.bed.gz



In [ ]:
bedtools intersect -c \
    -a GRCh37_AllTandemRepeats_gt100bp_slop5.bed \
    -b HG002v11-align2-GRCh37.dip.vcf \
    | awk '$4>1' \
    > HG002_GRCh37_MRG_stratification_ComplexVar_in_TR.bed

### Software versions

Python 2.7.15 |Anaconda, Inc.| (default, Nov 13 2018, 17:07:45)   
bedtools v2.27.1  
tabix (htslib) 1.7  
bgzip (htslib) 1.7  
